In [1]:
import csv

import numpy as np
import pandas as pd

from bokeh.plotting import output_notebook, show, figure

from sklearn.metrics import classification_report, accuracy_score
from sklearn.cross_validation import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Activation

output_notebook()

Using Theano backend.


Loading BokehJS ...

In [2]:
label_colors = {0: 'green', 1: 'red'}

In [3]:
features_by_axis = [
    'radius',
    'texture',
    'perimeter',
    'area',
    'smoothness',
    'compactness',
    'concavity',
    'concave points',
    'symmetry',
    'fractal_dimension',
]

columns = ['case', 'type']
input_columns = []
for axis in 'xyz':
    for feature in features_by_axis:
        feature_name = '{}_{}'.format(axis, feature)
        columns.append(feature_name)
        input_columns.append(feature_name)

label_column = 'type'

In [4]:
data = pd.read_csv('wdbc.data', names=columns)
data['type'] = data['type'].map(lambda x: 1 if x == 'M' else 0)
print(data.shape)

(569, 32)


In [5]:
feature1 = 'x_area'
feature2 = 'y_area'

colors = data[label_column].map(lambda x: label_colors[x])

f = figure()

f.xaxis.axis_label = "feature " + str(feature1)
f.yaxis.axis_label = "feature " + str(feature2)

f.circle(data[feature1], 
         data[feature2], 
         color=colors)
show(f)

In [6]:
train_data, test_data =  train_test_split(data, train_size=0.8)

In [7]:
model = Sequential([
    Dense(100, input_dim=len(input_columns)),
    Activation('tanh'),
    Dense(100),
    Activation('tanh'),
    Dense(1),
    Activation('sigmoid'),
])

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)

model.fit(
    train_data[input_columns].values, 
    train_data[label_column].values, 
    nb_epoch=100,
    batch_size=32,
)

Epoch 1/100
455/455 [==============================] - 0s - loss: 0.5322 - acc: 0.7934     
Epoch 2/100
455/455 [==============================] - 0s - loss: 0.3651 - acc: 0.8703     
Epoch 3/100
455/455 [==============================] - 0s - loss: 0.3106 - acc: 0.8879     
Epoch 4/100
455/455 [==============================] - 0s - loss: 0.2746 - acc: 0.8769     
Epoch 5/100
455/455 [==============================] - 0s - loss: 0.2899 - acc: 0.8967     
Epoch 6/100
455/455 [==============================] - 0s - loss: 0.2539 - acc: 0.9033     
Epoch 7/100
455/455 [==============================] - 0s - loss: 0.2737 - acc: 0.9033     
Epoch 8/100
455/455 [==============================] - 0s - loss: 0.2345 - acc: 0.9011     
Epoch 9/100
455/455 [==============================] - 0s - loss: 0.2556 - acc: 0.8923     
Epoch 10/100
455/455 [==============================] - 0s - loss: 0.2519 - acc: 0.8989     
Epoch 11/100
455/455 [==============================] - 0s - loss: 0.2419 - acc

In [8]:
train_predictions = model.predict(train_data[input_columns].values)
test_predictions = model.predict(test_data[input_columns].values)

sets = (
    ('train', train_data[label_column].values, train_predictions),
    ('test', test_data[label_column].values, test_predictions),
)

for name, labels, predictions in sets:
    predictions_as_int = np.rint(predictions)
    print('#' * 50)
    print(name)
    print('accuracy', accuracy_score(labels, predictions_as_int))
    print(classification_report(labels, predictions_as_int))
    print('predictions', dict(pd.DataFrame(predictions_as_int)[0].value_counts().items()))
    print()


##################################################
train
accuracy 0.916483516484
             precision    recall  f1-score   support

          0       0.89      0.98      0.93       277
          1       0.97      0.81      0.88       178

avg / total       0.92      0.92      0.91       455

predictions {0.0: 305, 1.0: 150}

##################################################
test
accuracy 0.964912280702
             precision    recall  f1-score   support

          0       0.95      1.00      0.98        80
          1       1.00      0.88      0.94        34

avg / total       0.97      0.96      0.96       114

predictions {0.0: 84, 1.0: 30}

